# Recuperacion de la Informaicon
# Examen Segundo Bimestre RI

**Nombre:** Paul Lora

In [37]:
%pip install rank_bm25

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [40]:
# imports

from sklearn.feature_extraction.text import TfidfVectorizer
import json
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import faiss
from openai import OpenAI
from dotenv import load_dotenv
import os
from rank_bm25 import BM25Okapi
import numpy as np



## Corpus

In [21]:
corpus = []
titles = []
identifiers = []
with open("../data/examen/data.json", "r") as f:
    for line in f:
        if line.strip():
            entry = json.loads(line)
            title = entry.get("title", "")
            abstract = entry.get("abstract", "")
            identifier = entry.get("id", "")
            corpus.append(abstract)
            titles.append(title)
            identifiers.append(identifier)

In [22]:
corpus_df = pd.DataFrame({
    "identifier": identifiers,
    "title": titles,
    "raw": corpus,
})

In [23]:
# preprocess function
stop_words = set(stopwords.words('english'))
def preprocess_doc(doc):
    words = word_tokenize(doc.lower())
    words_filtered = [word for word in words if word not in stop_words and word.isalpha()]
    return ' '.join(words_filtered)

In [24]:
corpus_df['processed'] = corpus_df['raw'].apply(preprocess_doc)
corpus_df['processed'] = corpus_df['processed'].str.replace("\n", " ")
corpus_df['processed'] = corpus_df['processed'].str.replace(r'\s+', ' ', regex=True).str.strip()
corpus_df

,identifier,title,raw,processed
0,0704.0001,Calculation of prompt diphoton production cros...,A fully differential calculation in perturba...,fully differential calculation perturbative qu...
1,0704.0002,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-...",describe new algorithm k game colors use obtai...
2,0704.0003,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is descri...,evolution system described dark matter field f...
3,0704.0004,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle...,show determinant stirling cycle numbers counts...
4,0704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\L...,paper show compute norm using dyadic grid resu...
...,...,...,...,...
27918,0710.0971,The Extending for Composite Skyrme Model,"In this paper, we have extended the composit...",paper extended composite skyrme model proposed...
27919,0710.0972,A Floer homology for exact contact embeddings,In this paper we construct the Floer homolog...,paper construct floer homology action function...
27920,0710.0973,Modulation invariant bilinear T(1) theorem,We prove a T(1) theorem for bilinear singula...,prove theorem bilinear singular integral opera...
27921,0710.0974,Hawking radiation in GHS and non-extremal D1-D...,We apply the method of Banerjee and Kulkarni...,apply method banerjee kulkarni provide derivat...


## TFIDF

In [25]:
corpus_processed_list = corpus_df['processed'].tolist()

In [26]:
tfidf_vectorizer=TfidfVectorizer(use_idf=True)
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(corpus_processed_list)

### Indexacion TFIDF

In [27]:
df = pd.DataFrame(tfidf_vectorizer_vectors.T.todense(), index=tfidf_vectorizer.get_feature_names_out())
df

,0,1,2,3,4,5,6,7,8,9,...,27913,27914,27915,27916,27917,27918,27919,27920,27921,27922
aa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aab,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aac,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aadl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aadt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zzgam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zzgamma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zzh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def search_tfidf(query, top_k=10):
    vertorized_query = tfidf_vectorizer.transform([query])
    cosine_similarities = cosine_similarity(vertorized_query, tfidf_vectorizer_vectors).flatten()
    top_indices = cosine_similarities.argsort()[-top_k:][::-1]
    return corpus_df.iloc[top_indices]['raw'].tolist()

## BM25

In [ ]:
bm25_corpus = [text.split() for text in corpus_df["raw"]] 
bm25 = BM25Okapi(bm25_corpus)

def search_bm25(query, top_k=10):
    query_tokens = preprocess_doc(query)
    scores = bm25.get_scores(query_tokens)
    top_indices = np.argsort(scores)[-top_k:][::-1]
    return corpus_df.iloc[top_indices]['raw'].tolist()

## Faiss

In [29]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(corpus_df["processed"].tolist(), convert_to_numpy=True)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

corpus_df['faiss_embeddings'] = embeddings.tolist()
corpus_df

,identifier,title,raw,processed,faiss_embeddings
0,0704.0001,Calculation of prompt diphoton production cros...,A fully differential calculation in perturba...,fully differential calculation perturbative qu...,"[-0.13706283271312714, 0.012870294973254204, 0..."
1,0704.0002,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-...",describe new algorithm k game colors use obtai...,"[0.003494551870971918, 0.07694141566753387, -0..."
2,0704.0003,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is descri...,evolution system described dark matter field f...,"[-0.019724154844880104, -0.05407233163714409, ..."
3,0704.0004,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle...,show determinant stirling cycle numbers counts...,"[-0.011817766353487968, 0.011769413948059082, ..."
4,0704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\L...,paper show compute norm using dyadic grid resu...,"[0.01388106681406498, 0.02092020958662033, -0...."
...,...,...,...,...,...
27918,0710.0971,The Extending for Composite Skyrme Model,"In this paper, we have extended the composit...",paper extended composite skyrme model proposed...,"[-0.06210463121533394, -0.0521075539290905, -0..."
27919,0710.0972,A Floer homology for exact contact embeddings,In this paper we construct the Floer homolog...,paper construct floer homology action function...,"[-0.07222992926836014, 0.008860399946570396, 0..."
27920,0710.0973,Modulation invariant bilinear T(1) theorem,We prove a T(1) theorem for bilinear singula...,prove theorem bilinear singular integral opera...,"[-0.0074160280637443066, -0.012459847144782543..."
27921,0710.0974,Hawking radiation in GHS and non-extremal D1-D...,We apply the method of Banerjee and Kulkarni...,apply method banerjee kulkarni provide derivat...,"[-0.013154637068510056, 0.04781310260295868, 0..."


In [ ]:
def search_faiss(index, query, top_k=10):
    query_embedding = model.encode([query], convert_to_numpy=True)
    _, indices = index.search(query_embedding, top_k)
    return corpus_df.iloc[indices.flatten()]['raw'].tolist()

## RAG

In [46]:
query = "Computer"

In [ ]:
contexto_tfidf = search_tfidf(query, top_k=5)
contexto_bm25 = search_bm25(query, top_k=5)
contexto_faiss = search_faiss(index, query, top_k=5)


In [48]:
print(contexto_bm25)

['  We study the flavor-changing-neutral-current process c to u mu+ mu- using 1.3\nfb^-1 of p p bar collisions at sqrt(s) = 1.96 TeV recorded by the D0 detector\noperating at the Fermilab Tevatron Collider. We see clear indications of the\nDs+ and D+ to phi pi+ to mu+ mu- pi+ final states with significance greater\nthan four standard deviations above background for the D+ state. We search for\nthe continuum decay of D+ to pi+mu+mu- in the dimuon invariant mass spectrum\naway from the phi resonance. We see no evidence of signal above background and\nset a limit of B(D+ to pi+mu+mu-) < 3.9 x 10^-6 at the 90% C.L. This limit\nplaces the most stringent constraint on new phenomena in the c to u mu+ mu-\ntransition.\n', '  A path from s to t on a polyhedral terrain is descending if the height of a\npoint p never increases while we move p along the path from s to t. No\nefficient algorithm is known to find a shortest descending path (SDP) from s to\nt in a polyhedral terrain. We give a simple

In [33]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)


### Respuesta TFIDF

In [49]:
prompt = f"""
Eres una aplicacion de Retrieval Augmented Generation que siempre responde en español.
Usa el siguiente contexto para responder a la pregunta del usuario.
Si la respuesta no se encuentra en el contexto, responde "No tengo suficiente información para responder a esa pregunta".

Contexto:
{contexto_tfidf}

Pregunta:
El usuario esta preguntando sobre "{query}".
"""

response = client.responses.create(
    model="gpt-4.1-mini",
    input=prompt
)
print(response.output_text)

El término "computer" en el contexto proporcionado se refiere principalmente a dos tipos: la computadora clásica y la computadora cuántica. 

Una computadora cuántica es una máquina que puede realizar ciertos cálculos mucho más rápido que una computadora clásica gracias a las leyes de la mecánica cuántica. Sin embargo, estas computadoras aún no existen plenamente, ya que es extremadamente difícil controlar sistemas mecánico-cuánticos con el grado necesario. Se considera que tecnologías maduras de procesamiento de información cuántica podrían utilizar luz (fotones) como portadores ideales de información cuántica.

Además, se menciona un concepto llamado "computadora de dualidad" que es una computadora cuántica en movimiento que aprovecha la dualidad onda-partícula y ofrece un poder adicional de cómputo mediante un paralelismo llamado "dualidad paralelismo". Por ejemplo, una computadora de dualidad con n-dubits puede ser modelada por una computadora cuántica con n+1 qubits.

Finalmente, 

In [53]:
prompt = f"""
Eres una aplicacion de Retrieval Augmented Generation que siempre responde en español.
Usa el siguiente contexto para responder a la pregunta del usuario.
Si la respuesta no se encuentra en el contexto, responde "No tengo suficiente información para responder a esa pregunta".

Contexto:
{contexto_bm25}

Pregunta:
El usuario esta preguntando sobre "{query}".
"""

response = client.responses.create(
    model="gpt-4.1-mini",
    input=prompt
)
print(response.output_text)

No tengo suficiente información para responder a esa pregunta.


### Respuesta FAISS

In [52]:
prompt = f"""
Eres una aplicacion de Retrieval Augmented Generation que siempre responde en español.
Usa el siguiente contexto para responder a la pregunta del usuario.
Si la respuesta no se encuentra en el contexto, responde "No tengo suficiente información para responder a esa pregunta".

Contexto:
{contexto_faiss}

Pregunta:
El usuario esta preguntando sobre "{query}".
"""

response = client.responses.create(
    model="gpt-4.1-mini",
    input=prompt
)
print(response.output_text)

El contexto proporciona varias referencias al uso de computadoras en diferentes áreas, como la adquisición de datos para experimentos de física, la importancia ética en la investigación en ciencias de la computación, la visualización gráfica para tareas científicas y de ingeniería, y la precisión limitada en cálculos numéricos científicos usando computadoras. 

Si te refieres a algún aspecto específico sobre "Computer" o computadoras, por favor indícame para poder ayudarte mejor con la información disponible.


## Evaluacion

### Documentos presentados por tfidf y faiss

In [ ]:
def get_relevant_docs_df(similar_docs):
    relevant_df = corpus_df[corpus_df['raw'].isin(similar_docs)][['identifier', 'title', 'raw']]
    relevant_df = relevant_df.rename(columns={'raw': 'abstract'})
    return relevant_df

relevant_tfidf_df = get_relevant_docs_df(contexto_tfidf)
relevant_bm25_df = get_relevant_docs_df(contexto_bm25)
relevant_faiss_df = get_relevant_docs_df(contexto_faiss)

print(relevant_tfidf_df)
print(relevant_faiss_df)

      identifier                                              title  \
1216   0704.1217                The Manin conjecture in dimension 2   
7283   0705.3281          Del Pezzo singularities and SUSY breaking   
10293  0706.1607  Partitions with independent iterates in random...   
23264  0709.0999  Classification of Toric log Del Pezzo Surfaces...   
25858  0709.3593  Noncommutative del Pezzo surfaces and Calabi-Y...   

                                                abstract  
1216     These lecture notes describe the current sta...  
7283     An analytic construction of compact Calabi-Y...  
10293    Consider an invertible measure-preserving tr...  
23264    Toric log Del Pezzo surfaces with Picard num...  
25858    The hypersurface in a 3-dimensional vector s...  
      identifier                                              title  \
5155   0705.1153  Optical Sky Brightness at Cerro Tololo Inter-A...   
12307  0706.3621                     Climate Change: The Sun's Role   
12406 